# Distribution shift / leakage checks

Goal: detect train vs test drift via a classifier, and identify which features
shift most (missingness and distribution).

In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

ROOT = Path("..").resolve()
DATA_DIR = ROOT / "data"

X_train = pd.read_csv(DATA_DIR / "training_set_features.csv", index_col="respondent_id")
X_test = pd.read_csv(DATA_DIR / "test_set_features.csv", index_col="respondent_id")

X_train = X_train.copy()
X_test = X_test.copy()
X_train["__is_train__"] = 1
X_test["__is_train__"] = 0

X_full = pd.concat([X_train, X_test], axis=0)
y_drift = X_full.pop("__is_train__")

cat_cols = X_full.select_dtypes(include=["object"]).columns.tolist()
for c in cat_cols:
    X_full[c] = X_full[c].astype("category")

params = dict(
    objective="binary",
    n_estimators=400,
    learning_rate=0.05,
    num_leaves=63,
    min_data_in_leaf=50,
    feature_fraction=0.9,
    bagging_fraction=0.9,
    bagging_freq=1,
    random_state=42,
)

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
oof = pd.Series(index=X_full.index, dtype=float)

for tr_idx, va_idx in skf.split(X_full, y_drift):
    X_tr, X_va = X_full.iloc[tr_idx], X_full.iloc[va_idx]
    y_tr, y_va = y_drift.iloc[tr_idx], y_drift.iloc[va_idx]
    model = lgb.LGBMClassifier(**params)
    model.fit(
        X_tr,
        y_tr,
        eval_set=[(X_va, y_va)],
        eval_metric="auc",
        categorical_feature=cat_cols,
        callbacks=[lgb.early_stopping(50, verbose=False)],
    )
    oof.iloc[va_idx] = model.predict_proba(X_va)[:, 1]

drift_auc = roc_auc_score(y_drift, oof)
print("Drift AUC (train vs test):", drift_auc)


C:\Users\Admin\AppData\Local\Temp\ipykernel_3104\3838542703.py:23: Pandas4Warning: For backward compatibility, 'str' dtypes are included by select_dtypes when 'object' dtype is specified. This behavior is deprecated and will be removed in a future version. Explicitly pass 'str' to `include` to select them, or to `exclude` to remove them and silence this warning.
See https://pandas.pydata.org/docs/user_guide/migration-3-strings.html#string-migration-select-dtypes for details on how to write code that works with pandas 2 and 3.
  cat_cols = X_full.select_dtypes(include=["object"]).columns.tolist()


[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Info] Number of positive: 21365, number of neg

## Feature shift ranking (missingness + numeric mean shift + categorical top share shift)

In [2]:
def shift_score_numeric(a: pd.Series, b: pd.Series) -> float:
    a = a.astype(float)
    b = b.astype(float)
    diff = abs(a.mean() - b.mean())
    scale = a.std() + b.std() + 1e-9
    return float(diff / scale)


def shift_score_categorical(a: pd.Series, b: pd.Series) -> float:
    top = pd.concat([a, b]).value_counts(dropna=True).idxmax()
    a_share = float((a == top).mean())
    b_share = float((b == top).mean())
    return abs(a_share - b_share)


rows = []
for col in X_train.columns:
    a = X_train[col]
    b = X_test[col]
    miss = abs(a.isna().mean() - b.isna().mean())
    if col in cat_cols:
        score = shift_score_categorical(a, b)
        ftype = "categorical"
    else:
        score = shift_score_numeric(a, b)
        ftype = "numeric"
    rows.append({"feature": col, "type": ftype, "shift_score": score, "missing_delta": miss})

shift_df = pd.DataFrame(rows)
shift_df = shift_df.sort_values(["shift_score", "missing_delta"], ascending=False)
shift_df.head(15)

,feature,type,shift_score,missing_delta
35,__is_train__,numeric,1.000000e+09,0.000000e+00
14,health_insurance,numeric,1.278792e-02,1.739538e-03
30,census_msa,categorical,1.050487e-02,0.000000e+00
20,opinion_seas_sick_from_vacc,numeric,9.460677e-03,5.998243e-04
6,behavioral_large_gatherings,numeric,7.442953e-03,5.617514e-04
8,behavioral_touch_face,numeric,6.952068e-03,1.794500e-07
12,child_under_6_months,numeric,6.690212e-03,2.632434e-04
16,opinion_h1n1_risk,numeric,6.140875e-03,3.000797e-04
31,household_adults,numeric,5.180328e-03,8.989562e-04
32,household_children,numeric,4.917228e-03,8.989562e-04
